In [1]:
!pip install -q --upgrade pip

## installing the latest transformers version from pip
!pip install --use-feature=2020-resolver -q transformers==3.0.2
import transformers

## installing Google Translator package
!pip install -q googletrans

     |████████████████████████████████| 1.5MB 5.2MB/s 
     |████████████████████████████████| 769 kB 6.4 MB/s 
     |████████████████████████████████| 883 kB 14.7 MB/s 
     |████████████████████████████████| 1.1 MB 19.3 MB/s 
     |████████████████████████████████| 3.0 MB 24.8 MB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 981 kB 7.8 MB/s 
     |████████████████████████████████| 42 kB 949 kB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 98 kB 5.1 MB/s 


In [2]:
import gc
import os
import random
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K

from googletrans import Translator
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

TensorFlow version: 2.3.0
Transformers version: 3.0.2


In [17]:
class Configuration():
    """
    All configuration for running an experiment
    """
    def __init__(
        self,
        model_name,
        truncation = True,
        translation = True,
        max_length = 64,
        padding = True,
        batch_size = 128,
        epochs = 5,
        learning_rate = 1e-5,
        metrics = ["sparse_categorical_accuracy"],
        verbose = 1,
        train_splits = 5,
        accelerator = "TPU",
        myluckynumber = 13
    ):
        # seed and accelerator
        self.SEED = myluckynumber
        self.ACCELERATOR = accelerator

        # paths
        self.PATH_TRAIN = Path("/content/sample_data/new_train.csv")
        self.PATH_TEST  = Path("/content/sample_data/test.csv")

        # splits
        self.TRAIN_SPLITS = train_splits

        
        # model configuration
        self.MODEL_NAME = model_name
        # self.TRANSLATION = translation
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_NAME)

        # model hyperparameters
        self.MAX_LENGTH = max_length
        self.PAD_TO_MAX_LENGTH = padding
        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.LEARNING_RATE = learning_rate
        self.METRICS = metrics
        self.VERBOSE = verbose
        
        # initializing accelerator
        self.initialize_accelerator()

    def initialize_accelerator(self):
        """
        Initializing accelerator
        """
        # checking TPU first
        if self.ACCELERATOR == "TPU":
            print("Connecting to TPU")
            try:
                tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
                print(f"Running on TPU {tpu.master()}")
            except ValueError:
                print("Could not connect to TPU")
                tpu = None

            if tpu:
                try:
                    print("Initializing TPU")
                    tf.config.experimental_connect_to_cluster(tpu)
                    tf.tpu.experimental.initialize_tpu_system(tpu)
                    self.strategy = tf.distribute.experimental.TPUStrategy(tpu)
                    self.tpu = tpu
                    print("TPU initialized")
                except _:
                    print("Failed to initialize TPU")
            else:
                print("Unable to initialize TPU")
                self.ACCELERATOR = "GPU"

        # default for CPU and GPU
        if self.ACCELERATOR != "TPU":
            print("Using default strategy for CPU and single GPU")
            self.strategy = tf.distribute.get_strategy()

        # checking GPUs
        if self.ACCELERATOR == "GPU":
            print(f"GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

        # defining replicas
        self.AUTO = tf.data.experimental.AUTOTUNE
        self.REPLICAS = self.strategy.num_replicas_in_sync
        print(f"REPLICAS: {self.REPLICAS}")

In [3]:
def encode_text(df, tokenizer, max_len, padding):
    """
    Preprocessing textual data into encoded tokens.
    """
    text = df["text"].values.tolist()

    # encoding text using tokenizer of the model
    text_encoded = tokenizer.batch_encode_plus(
        text,
        pad_to_max_length = padding,
        max_length = max_len
    )

    return text_encoded


def get_tf_dataset(X, y, auto, labelled = True, repeat = False, shuffle = False, batch_size = 128):
    """
    Creating tf.data.Dataset for TPU.
    """
    if labelled:
        ds = (tf.data.Dataset.from_tensor_slices((X["input_ids"], y)))
    else:
        ds = (tf.data.Dataset.from_tensor_slices(X["input_ids"]))

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(2048)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(auto)

    return ds

In [4]:
def build_model(model_name, max_len, learning_rate, metrics):
    """
    Building the Deep Learning architecture
    """
    # defining encoded inputs
    input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
    
    # defining transformer model embeddings
    transformer_model = TFAutoModel.from_pretrained(model_name)
    transformer_embeddings = transformer_model(input_ids)[0]

    # defining output layer
    output_values = Dense(3, activation = "softmax")(transformer_embeddings[:, 0, :])

    # defining model
    model = Model(inputs = input_ids, outputs = output_values)
    opt = Adam(learning_rate = learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    metrics = metrics

    model.compile(optimizer = opt, loss = loss, metrics = metrics)

    return model

In [6]:
def run_model(config):
    """
    Running the model
    """
    ## reading data
    df_train = pd.read_csv(config.PATH_TRAIN)
    df_test = pd.read_csv(config.PATH_TEST)

     # stratified K-fold on language and label
    skf = StratifiedKFold(n_splits = config.TRAIN_SPLITS, shuffle = True, random_state = config.SEED)

    # initializing predictions
    preds_oof = np.zeros((df_train.shape[0], 3))
    preds_test = np.zeros((df_test.shape[0], 3))
    acc_oof = []

    # iterating over folds
    for (fold, (train_index, valid_index)) in enumerate(skf.split(df_train, df_train.author)):
        # initializing TPU
        if config.ACCELERATOR == "TPU":
            if config.tpu:
                config.initialize_accelerator()

        # building model
        K.clear_session()
        with config.strategy.scope():
            model = build_model(config.MODEL_NAME, config.MAX_LENGTH, config.LEARNING_RATE, config.METRICS)
            if fold == 0:
                print(model.summary())

        print("\n")
        print("#" * 19)
        print(f"##### Fold: {fold + 1} #####")
        print("#" * 19)

        # splitting data into training and validation
        X_train = df_train.iloc[train_index]
        X_valid = df_train.iloc[valid_index]

        y_train = X_train.author.values
        y_valid = X_valid.author.values

        print("\nTokenizing")

        # encoding text data using tokenizer
        X_train_encoded = encode_text(df = X_train, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)
        X_valid_encoded = encode_text(df = X_valid, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # creating TF Dataset
        ds_train = get_tf_dataset(X_train_encoded, y_train, config.AUTO, repeat = True, shuffle = True, batch_size = config.BATCH_SIZE * config.REPLICAS)
        ds_valid = get_tf_dataset(X_valid_encoded, y_valid, config.AUTO, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        n_train = X_train.shape[0]

        if fold == 0:
            X_test_encoded = encode_text(df = df_test, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # saving model at best accuracy epoch
        sv = tf.keras.callbacks.ModelCheckpoint(
            "model.h5",
            monitor = "val_sparse_categorical_accuracy",
            verbose = 0,
            save_best_only = True,
            save_weights_only = True,
            mode = "max",
            save_freq = "epoch"
        )

        print("\nTraining")

        # training model
        model_history = model.fit(
            ds_train,
            epochs = config.EPOCHS,
            callbacks = [sv],
            steps_per_epoch = n_train / config.BATCH_SIZE // config.REPLICAS,
            validation_data = ds_valid,
            verbose = config.VERBOSE
        )

        print("\nValidating")

        # scoring validation data
        model.load_weights("model.h5")
        ds_valid = get_tf_dataset(X_valid_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        preds_valid = model.predict(ds_valid, verbose = config.VERBOSE)
        acc = accuracy_score(y_valid, np.argmax(preds_valid, axis = 1))

        preds_oof[valid_index] = preds_valid
        acc_oof.append(acc)

        print("\nInferencing")

        # scoring test data
        ds_test = get_tf_dataset(X_test_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)
        preds_test += model.predict(ds_test, verbose = config.VERBOSE) / config.TRAIN_SPLITS

        print(f"\nFold {fold + 1} Accuracy: {round(acc, 4)}\n")

        g = gc.collect()

    # overall CV score and standard deviation
    print(f"\nCV Mean Accuracy: {round(np.mean(acc_oof), 4)}")
    print(f"CV StdDev Accuracy: {round(np.std(acc_oof), 4)}\n")

    return preds_oof, preds_test

In [ ]:
# Model: Bert Base Cased
# config_1 = Configuration("bert-base-cased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_1, preds_test_1 = run_model(config_1)

# Model: Bert Base Uncased
# config_2 = Configuration("bert-base-uncased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_2, preds_test_2 = run_model(config_2)


In [ ]:
# Model: Bert Large Cased
# config_3 = Configuration("bert-large-cased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_3, preds_test_3 = run_model(config_3)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at bert-large-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-large-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 32, 1024), (None, 333579264 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 333,582,339
Trainable params: 333,582,339
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 287s 942ms/step - loss: 0.8463 - sparse_categorical_accuracy: 0.7012 - val_loss: 0.7645 - val_sparse_categorical_accuracy: 0.7781
Epoch 2/2
305/305 [==============================] - 283s 929ms/step - loss: 0.7092 - sparse_categorical_accuracy: 0.8379 - val_loss: 0.7347 - val_sparse_categorical_accuracy: 0.8127

Validating
77/77 [==============================] - 64s 826ms/step

Inferencing
66/66 [==============================] - 54s 815ms/step

Fold 1 Accuracy: 0.8127



Some weights of the model checkpoint at bert-large-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-large-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. 



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 288s 944ms/step - loss: 0.8390 - sparse_categorical_accuracy: 0.7036 - val_loss: 0.7783 - val_sparse_categorical_accuracy: 0.7645
Epoch 2/2
305/305 [==============================] - 283s 927ms/step - loss: 0.7007 - sparse_categorical_accuracy: 0.8478 - val_loss: 0.7344 - val_sparse_categorical_accuracy: 0.8114

Validating
77/77 [==============================] - 64s 827ms/step

Inferencing
66/66 [==============================] - 54s 815ms/step

Fold 2 Accuracy: 0.8114


CV Mean Accuracy: 0.812
CV StdDev Accuracy: 0.0006



In [ ]:
# Model: Bert Large Uncased
# config_4 = Configuration("bert-large-uncased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_4, preds_test_4 = run_model(config_4)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max leng

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 32, 1024), (None, 335141888 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 335,144,963
Trainable params: 335,144,963
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 288s 943ms/step - loss: 0.8580 - sparse_categorical_accuracy: 0.6860 - val_loss: 0.7647 - val_sparse_categorical_accuracy: 0.7785
Epoch 2/2
305/305 [==============================] - 283s 927ms/step - loss: 0.7253 - sparse_categorical_accuracy: 0.8218 - val_loss: 0.7460 - val_sparse_categorical_accuracy: 0.7996

Validating
77/77 [==============================] - 64s 826ms/step

Inferencing
66/66 [==============================] - 54s 816ms/step

Fold 1 Accuracy: 0.7996



Some weights of the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max leng



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 288s 943ms/step - loss: 0.8627 - sparse_categorical_accuracy: 0.6792 - val_loss: 0.7789 - val_sparse_categorical_accuracy: 0.7652
Epoch 2/2
305/305 [==============================] - 283s 926ms/step - loss: 0.7299 - sparse_categorical_accuracy: 0.8207 - val_loss: 0.7434 - val_sparse_categorical_accuracy: 0.8015

Validating
77/77 [==============================] - 64s 826ms/step

Inferencing
66/66 [==============================] - 54s 816ms/step

Fold 2 Accuracy: 0.8015


CV Mean Accuracy: 0.8006
CV StdDev Accuracy: 0.001



In [ ]:
# Model: Distilbert Base Cased
# config_6 = Configuration("distilbert-base-cased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_6, preds_test_6 = run_model(config_6)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a spec

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 32, 768),)        65190912  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 65,193,219
Trainable params: 65,193,219
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 49s 160ms/step - loss: 0.8918 - sparse_categorical_accuracy: 0.6511 - val_loss: 0.8100 - val_sparse_categorical_accuracy: 0.7350
Epoch 2/2
305/305 [==============================] - 47s 153ms/step - loss: 0.7747 - sparse_categorical_accuracy: 0.7718 - val_loss: 0.7854 - val_sparse_categorical_accuracy: 0.7577

Validating
77/77 [==============================] - 10s 129ms/step

Inferencing
66/66 [==============================] - 9s 130ms/step

Fold 1 Accuracy: 0.7577



Some weights of the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a spec



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 48s 159ms/step - loss: 0.8875 - sparse_categorical_accuracy: 0.6551 - val_loss: 0.8068 - val_sparse_categorical_accuracy: 0.7387
Epoch 2/2
305/305 [==============================] - 47s 153ms/step - loss: 0.7611 - sparse_categorical_accuracy: 0.7862 - val_loss: 0.7852 - val_sparse_categorical_accuracy: 0.7589

Validating
77/77 [==============================] - 10s 129ms/step

Inferencing
66/66 [==============================] - 9s 129ms/step

Fold 2 Accuracy: 0.7589


CV Mean Accuracy: 0.7583
CV StdDev Accuracy: 0.0006



In [ ]:
# Model: Distilbert Base Uncased
# config_7 = Configuration("distilbert-base-uncased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_7, preds_test_7 = run_model(config_7)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 32, 768),)        66362880  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 66,365,187
Trainable params: 66,365,187
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 49s 159ms/step - loss: 0.8620 - sparse_categorical_accuracy: 0.6869 - val_loss: 0.7784 - val_sparse_categorical_accuracy: 0.7674
Epoch 2/2
305/305 [==============================] - 47s 154ms/step - loss: 0.7469 - sparse_categorical_accuracy: 0.8020 - val_loss: 0.7525 - val_sparse_categorical_accuracy: 0.7936

Validating
77/77 [==============================] - 10s 129ms/step

Inferencing
66/66 [==============================] - 9s 129ms/step

Fold 1 Accuracy: 0.7936



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a 



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 48s 159ms/step - loss: 0.8740 - sparse_categorical_accuracy: 0.6653 - val_loss: 0.7908 - val_sparse_categorical_accuracy: 0.7536
Epoch 2/2
305/305 [==============================] - 47s 153ms/step - loss: 0.7487 - sparse_categorical_accuracy: 0.7984 - val_loss: 0.7577 - val_sparse_categorical_accuracy: 0.7871

Validating
77/77 [==============================] - 10s 129ms/step

Inferencing
66/66 [==============================] - 9s 129ms/step

Fold 2 Accuracy: 0.7871


CV Mean Accuracy: 0.7903
CV StdDev Accuracy: 0.0032



In [ ]:
# Model: Roberta Base
# config_9 = Configuration("roberta-base", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_9, preds_test_9 = run_model(config_9)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulti

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 32, 768), (None,  124645632 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 124,647,939
Trainable params: 124,647,939
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 96s 315ms/step - loss: 0.9155 - sparse_categorical_accuracy: 0.6164 - val_loss: 0.7852 - val_sparse_categorical_accuracy: 0.7597
Epoch 2/2
305/305 [==============================] - 92s 301ms/step - loss: 0.7729 - sparse_categorical_accuracy: 0.7723 - val_loss: 0.7853 - val_sparse_categorical_accuracy: 0.7573

Validating
77/77 [==============================] - 20s 256ms/step

Inferencing
66/66 [==============================] - 17s 256ms/step

Fold 1 Accuracy: 0.7597



Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulti



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 96s 314ms/step - loss: 0.9071 - sparse_categorical_accuracy: 0.6246 - val_loss: 0.7906 - val_sparse_categorical_accuracy: 0.7509
Epoch 2/2
305/305 [==============================] - 93s 306ms/step - loss: 0.7658 - sparse_categorical_accuracy: 0.7803 - val_loss: 0.7664 - val_sparse_categorical_accuracy: 0.7780

Validating
77/77 [==============================] - 20s 256ms/step

Inferencing
66/66 [==============================] - 17s 258ms/step

Fold 2 Accuracy: 0.778


CV Mean Accuracy: 0.7688
CV StdDev Accuracy: 0.0092



In [ ]:
# Model: Roberta Large
# config_10 = Configuration("roberta-large", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_10, preds_test_10 = run_model(config_10)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaul

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 32, 1024), (None, 355359744 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 355,362,819
Trainable params: 355,362,819
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 291s 953ms/step - loss: 0.9695 - sparse_categorical_accuracy: 0.5538 - val_loss: 0.8819 - val_sparse_categorical_accuracy: 0.6565
Epoch 2/2
305/305 [==============================] - 286s 937ms/step - loss: 0.8217 - sparse_categorical_accuracy: 0.7225 - val_loss: 0.7766 - val_sparse_categorical_accuracy: 0.7685

Validating
77/77 [==============================] - 63s 821ms/step

Inferencing
66/66 [==============================] - 54s 811ms/step

Fold 1 Accuracy: 0.7685



Some weights of the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaul



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 291s 955ms/step - loss: 0.8996 - sparse_categorical_accuracy: 0.6343 - val_loss: 0.8114 - val_sparse_categorical_accuracy: 0.7325
Epoch 2/2
305/305 [==============================] - 286s 937ms/step - loss: 0.7837 - sparse_categorical_accuracy: 0.7624 - val_loss: 0.7563 - val_sparse_categorical_accuracy: 0.7904

Validating
77/77 [==============================] - 63s 821ms/step

Inferencing
66/66 [==============================] - 54s 812ms/step

Fold 2 Accuracy: 0.7904


CV Mean Accuracy: 0.7795
CV StdDev Accuracy: 0.0109



In [ ]:
# Model: XLM Roberta Base
# config_11 = Configuration("jplu/tf-xlm-roberta-base", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
# preds_train_11, preds_test_11 = run_model(config_11)


Connecting to TPU
Could not connect to TPU
Unable to initialize TPU
Using default strategy for CPU and single GPU
GPUs Available: 1
REPLICAS: 1


Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 32, 768), (None,  278043648 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 278,045,955
Trainable params: 278,045,955
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 121s 398ms/step - loss: 0.9793 - sparse_categorical_accuracy: 0.5448 - val_loss: 0.9055 - val_sparse_categorical_accuracy: 0.6291
Epoch 2/2
305/305 [==============================] - 119s 389ms/step - loss: 0.8544 - sparse_categorical_accuracy: 0.6844 - val_loss: 0.8208 - val_sparse_categorical_accuracy: 0.7220

Validating
77/77 [==============================] - 20s 256ms/step

Inferencing
66/66 [==============================] - 17s 259ms/step

Fold 1 Accuracy: 0.722



Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples



###################
##### Fold: 2 #####
###################

Tokenizing


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Training
Epoch 1/2
305/305 [==============================] - 121s 397ms/step - loss: 1.0360 - sparse_categorical_accuracy: 0.4711 - val_loss: 0.9597 - val_sparse_categorical_accuracy: 0.5243
Epoch 2/2
305/305 [==============================] - 119s 391ms/step - loss: 0.9010 - sparse_categorical_accuracy: 0.6352 - val_loss: 0.8427 - val_sparse_categorical_accuracy: 0.6996

Validating
77/77 [==============================] - 20s 257ms/step

Inferencing
66/66 [==============================] - 17s 260ms/step

Fold 2 Accuracy: 0.6996


CV Mean Accuracy: 0.7108
CV StdDev Accuracy: 0.0112

